In [1]:
import numpy as np
from scipy.stats import entropy
from math import log, e
import pandas as pd
import math

In [2]:
medecin_gen = pd.read_csv('medecin_gen.csv')

In [11]:
def calc_entropy(column):
    counts = np.bincount(column)
    probability = counts / len(column)
    entropy = 0
    for prob in probability:
        if prob > 0:
            entropy += prob * math.log(prob, 2)

    n = len(counts)
    max_entropy = math.log2(n)  # Entropie maximale

    # Normalise l'entropie
    normalized_entropy = -entropy / max_entropy if max_entropy > 0 else 0

    return normalized_entropy

In [4]:
medecin_gen.drop(columns= ['Unnamed: 0'],inplace=True)

In [12]:
liste_var = ['Nb Skills', 'moyens_paiement',
       'carte_vitale', 'secteur',
       'site',
       'Diplômes nationaux et universitaires b', 'Nb caractères présentation',
       'Autres formations b','Travaux et publications b','Nb langues', 'Expériences b', 'Carac_card_1',
       'Transport', 'Parking', 'Infos', 'Coordonnées', 'Sans RDV', 'Formations b',
       'nb terms card_0', 'nb terms card_3', 'nb terms card_4',
       'nb carac card_0', 'nb carac card_3', 'nb carac card_4']
entropy={}
for nom_colonne in liste_var : 
    entropy[nom_colonne]=calc_entropy(medecin_gen[nom_colonne])
print(entropy)

{'Nb Skills': 0.5612618570483375, 'moyens_paiement': 0.17203694935311378, 'carte_vitale': 0.17203694935311378, 'secteur': 0.7193329040881774, 'site': 0.7793498372920852, 'Diplômes nationaux et universitaires b': 0.9881108365218301, 'Nb caractères présentation': 0.44967468197843274, 'Autres formations b': 0.8904916402194913, 'Travaux et publications b': 0.39124356362925566, 'Nb langues': 0.8405314861561367, 'Expériences b': 0.6193821946787638, 'Carac_card_1': 0.6056585055695921, 'Transport': 0.17203694935311378, 'Parking': 0.9881108365218301, 'Infos': -0.0, 'Coordonnées': 0.9881108365218301, 'Sans RDV': 0, 'Formations b': 0.9182958340544896, 'nb terms card_0': 0.6287140667996616, 'nb terms card_3': 0.6701958403955256, 'nb terms card_4': 0.6566052885484794, 'nb carac card_0': 0.31810835043837404, 'nb carac card_3': 0.3466625322728095, 'nb carac card_4': 0.3307771924431801}
